In [1]:
import pandas as pd
import numpy as np
import common as cm

This exercise consists of 3 parts. Finish the first part to get a mark of 3.0. The first two parts to get 4.0. Finish all exercies to get 5.0.

# Part 1: Concordance

1.1) Given are the following modes of transport: bus, bike, car, train. Each mode is characterized by 2 cost-type criteria: price and time; and 2 gain-type criteria: comfort and reliability.

Mode of transport | Time | Comfort | Price | Reliability
--- | --- | --- | --- | ---
 **bus**  | 6 | 3  | 6 | 2
 **bike** | 8 | 2  | 2 | 8
 **car**  | 2 | 10 | 9 | 7
 **train**| 3 | 5  | 5 | 6
 **by foot**| 10 | 2  | 0 | 5

In [2]:
data = {
    'mode': ['bus', 'bike', 'car', 'train', 'foot'],
    'time': [6, 8, 2, 3, 10],
    'comfort': [3, 2, 10, 5, 2],
    'price': [6, 2, 9, 5, 0],
    'reliability': [2, 8, 7, 6, 5]
}

criteria = ['time', 'comfort', 'price', 'reliability']
data = pd.DataFrame(data)

data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


1.2)  Below are the parameters, i.e., threholds, criterion-type, and weights, for each criterion,

In [3]:
parameters = {'time': {'weights': 4, 'q': 1.0, 'p': 2, 'v': 4, 'type': 'cost'},
 'comfort': {'weights': 2, 'q': 2.0, 'p': 3, 'v': 6, 'type': 'gain'},
 'price': {'weights': 3, 'q': 1.0, 'p': 3, 'v': 5, 'type': 'cost'},
 'reliability': {'weights': 1, 'q': 1.5, 'p': 3, 'v': 5, 'type': 'gain'}}
sum_weights = 10.

pd.DataFrame(parameters).T

,weights,q,p,v,type
time,4,1,2,4,cost
comfort,2,2,3,6,gain
price,3,1,3,5,cost
reliability,1,1.5,3,5,gain


1.3) Finish the below function for calculating a marginal concordance for $i$-th criterion (gain type) $c_i(g_i(a),g_i(b))$ based on q and p tresholds. 

In [4]:
def getConcordanceGain(gA, gB, q, p):
    if gB <= gA + q:
        return 1
    if gB <= gA + p:
        return (gB - gA - q)/(p - q)
    return 0
    
def getConcordanceCost(gA, gB, q, p):
    return getConcordanceGain(gB, gA, q, p)

1.4)  Calculate  comprehensive concordance  index  for  all  criteria  of  alternatives $a$ and $b$. Remember that comprehensive concordance must be divided by the sum of weights.

In [5]:
def getComprehensiveConcordance(A, B, criteria, parameters):
    concordance = 0.0
    weight_sum = 0.0
    for criterion in criteria:
        parameter = parameters[criterion]
        gA, gB = A[criterion], B[criterion]
        w, q, p, param_type = parameter['weights'], parameter['q'], parameter['p'], parameter['type']
        if param_type == 'gain':
            concordance += getConcordanceGain(gA, gB, q, p) * w
        else:
            concordance += getConcordanceCost(gA, gB, q, p) * w
        weight_sum += w
    concordance /= weight_sum
    return concordance

1.5) Check comprehensive concordance indexes for C(bus, some transportation):

In [6]:
for alternative_id, alternative_row in data.iterrows():
    print("C({0},{1}) = ".format(0, alternative_id), getComprehensiveConcordance(data.loc[0], alternative_row, criteria, parameters))

C(0,0) =  1.0
C(0,1) =  0.6
C(0,2) =  0.3
C(0,3) =  0.5
C(0,4) =  0.7


1.6) Finish the below function for generating a concordance matrix. Use a majority_threshold as a cutting-level. For hich pairs a concordance is fulfilled?

In [10]:
def getConcordanceMatrix(data, criteria, parameters, majority_threshold=0.7):
    n = len(data)
    concordance_matrix = np.zeros((n,n))
    for a in range(n):
        for b in range(n):
            A = data.loc[a, :]
            B = data.loc[b, :]
            concordance_matrix[a][b] = 1 if getComprehensiveConcordance(A, B, criteria, parameters) >= majority_threshold else 0     
    return concordance_matrix

In [11]:
print(getConcordanceMatrix(data, criteria, parameters))

[[1. 0. 0. 0. 1.]
 [1. 1. 0. 0. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [0. 1. 0. 0. 1.]]


# Part 2: outranking graph

2.1) Complete the function that calculates marginal discordance for  $i$-th  criterion (gain type)  $d_i(a,b)$  basing on v treshold. $d_i(a,b) = 1$ when $a$ is worse than $b$ on criterion $i$ by at least the veto threshold, zero otherwise.

In [12]:
def getDiscordanceGain(A ,B, v):
    if B >= A + v:
        return 1
    return 0

def getDiscordanceCost(gA, gB, v):
    return getDiscordanceGain(gB, gA, v)


2.2) Calculate a comprehensive discordance index.  $𝐷(a,b)=1$  if at least one criterion vetoes against $a S b$.

In [13]:
def getComprehensiveDiscordance(A, B, criteria, parameters):
    discordance = 0
    for criterion in criteria:
        gA, gB = A[criterion], B[criterion]
        parameter = parameters[criterion]
        param_type, v = parameter['type'], parameter['v']
        if param_type == 'gain':
            discordance = getDiscordanceGain(gA, gB, v)
        else:
            discordance = getDiscordanceCost(gA, gB, v)
        if discordance == 1:
            return 1
    return 0

2.3) Check comprehensive discordance indexes for D(bus, some transportation):

In [14]:
for alternative_id, alternative_row in data.iterrows():
    print("D({0},{1}) = ".format(0, alternative_id),getComprehensiveDiscordance(data.loc[0], alternative_row, criteria, parameters))

D(0,0) =  0
D(0,1) =  1
D(0,2) =  1
D(0,3) =  0
D(0,4) =  1


2.4) Finish the below function for calculating a comprehensive discordance matrix.

In [15]:
def getDiscordanceMatrix(data, criteria, parameters):
    n = len(data)
    discordance_matrix = np.zeros((n,n))
    
    for a in range(n):
        for b in range(n):
            A = data.loc[a, :]
            B = data.loc[b, :]
            discordance_matrix[a][b] = getComprehensiveDiscordance(A, B, criteria, parameters)
    return discordance_matrix

In [16]:
getDiscordanceMatrix(data, criteria, parameters)

array([[0., 1., 1., 0., 1.],
       [0., 0., 1., 1., 0.],
       [0., 1., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [1., 0., 1., 1., 0.]])

2.5) Now, finish the below function for generating the outranking matrixThis method should take into account both the concordance and discordance matrices.

In [17]:
def getOutrankingMatrix(data, criteria, parameters, majority_threshold = 0.7):
    n = len(data)
    outranking_matrix = np.zeros((n,n))
    concordance_matrix = getConcordanceMatrix(data, criteria, parameters, majority_threshold)
    discordance_matrix = getDiscordanceMatrix(data, criteria, parameters)
    for a in range(n):
        for b in range(n):
            if concordance_matrix[a][b] == 1 and discordance_matrix[a][b] == 0:
                outranking_matrix[a][b] = 1
    return outranking_matrix

In [18]:
outranking_matrix = getOutrankingMatrix(data, criteria, parameters, majority_threshold = 0.85)
print(outranking_matrix)

[[1. 0. 0. 0. 0.]
 [1. 1. 0. 0. 1.]
 [1. 0. 1. 0. 0.]
 [1. 1. 0. 1. 0.]
 [0. 1. 0. 0. 1.]]


2.3) Change outranking matrix to adjacency list as a dictionary that every alternative $a$ has a list of alternatives that $a$ outranks. For simplicity, assume that there are no edges between the vertex and itself (a->a).

In [19]:
data

,mode,time,comfort,price,reliability
0,bus,6,3,6,2
1,bike,8,2,2,8
2,car,2,10,9,7
3,train,3,5,5,6
4,foot,10,2,0,5


In [20]:
def toAdjacencyList(outranking_matrix, names = None):
    n = len(outranking_matrix)
    adjacency_list = dict()
    for a in range(n):
        name_a = a if names is None else names[a]
        adjacency_list[name_a] = list()
        for b in range(n):
            if b == a:
                continue
            if outranking_matrix[a][b] == 1:
                name_b = b if names is None else names[b]
                adjacency_list[name_a].append(name_b)
    return adjacency_list

In [21]:
graph = toAdjacencyList(outranking_matrix,  names = data['mode'])
print(graph)

{'bus': [], 'bike': ['bus', 'foot'], 'car': ['bus'], 'train': ['bus', 'bike'], 'foot': ['bike']}


2.4) Draw outranking graph, and remove cycles (manually).

In [52]:
cm.PrintGraph(graph, filename="graph_part_2")

2.5) Which mode of transport are in the kernel?

# Part 3: Kernel

3.1) Given is the below outranking matrix

In [132]:
outranking_matrix =np.array(   [[0., 1., 0., 0., 0., 0., 1., 0.],
                                [0., 0., 1., 0., 1., 0., 0., 0.],
                                [0., 0., 0., 0., 0., 0., 0., 1.],
                                [0., 0., 0., 0., 1., 0., 1., 0.],
                                [0., 0., 0., 0., 0., 1., 0., 0.],
                                [0., 0., 0., 0., 0., 0., 1., 0.],
                                [0., 0., 0., 0., 0., 0., 0., 1.],
                                [0., 0., 0., 0., 0., 0., 0., 0.]])

In [133]:
graph = toAdjacencyList(outranking_matrix)

In [134]:
graph

{0: [1, 6], 1: [2, 4], 2: [7], 3: [4, 6], 4: [5], 5: [6], 6: [7], 7: []}

3.2) Display the outranking graph. Which alternatives belong to kernel?

In [135]:
cm.PrintGraph(graph, filename="graph_part_3")

3.3) In this exercise, you are asked to complete the function for constructing a kernel. Firstly, complete the below auxiliary function which reverses edges of the graph.

In [136]:
def getReverseGraph(graph):
    reversed_graph = dict()
    for index in graph:
        reversed_graph[index] = list()
        for el in graph:
            if graph[el] == index:
                continue
            else:
                if index in graph[el]:
                    reversed_graph[index].append(el)
    return reversed_graph 

3.4) Verify the correctness: compare the below reverse_graph with the above graph.

In [137]:
reverse_graph = getReverseGraph(graph)
reverse_graph

{0: [], 1: [0], 2: [1], 3: [], 4: [1, 3], 5: [4], 6: [0, 3, 5], 7: [2, 6]}

In [138]:
cm.PrintGraph(reverse_graph, filename="reverse_graph_part_3")

3.5) Now, complete the below function for finding a graph kernel. This algorithm should proceed in the following way: <br>
1) Find non-outranked vertices. Add them to the kernel. <br> 
2) Remove vertices found in step 1 from the graph and these vertices which are directly surpassed by them.<br>
3) Repeat (go to 1) until all vertices are removed from the graph. 

In [139]:
def Kernel(graph):
    reverse_graph = getReverseGraph(graph)
    kernel = []
    remove = []
    while len(graph)>0:
        for a,b in reverse_graph.items():
            if len(b) == 0:
                remove.append(a)
                kernel.append(a)
                for u,t in graph.items():
                    if u == a:
                        for item in t:
                            if item not in remove:
                                remove.append(item)
        for rm in remove:
            del graph[rm]
        for h,g in graph.items():
            for rm in remove:
                if rm in g:
                    g.remove(rm)
        remove = []
        reverse_graph = getReverseGraph(graph)
        
    return sorted(kernel)

In [140]:
graph = toAdjacencyList(outranking_matrix)
Kernel(graph)

[0, 2, 3, 5]